In [ ]:
import helpers.hs3 as hs3
import helpers.hsql as hsql
import im_v2.common.data.client as icdc
import im_v2.im_lib_tasks as imvimlita
import market_data as mdata
import oms.ccxt_broker as occxbrok
import oms.oms_db as oomsdb
import oms.order_example as oordexam

## Functions

In [ ]:
def create_all_tables(connection):
    """
    Create submitted and accepted orders tables with default names.
    """
    incremental = False
    oomsdb.create_submitted_orders_table(
        connection, incremental, oomsdb.SUBMITTED_ORDERS_TABLE_NAME
    )
    oomsdb.create_accepted_orders_table(
        connection, incremental, oomsdb.ACCEPTED_ORDERS_TABLE_NAME
    )

## DB

### Connection

In [ ]:
# Get environment variables with login info.
env_file = imvimlita.get_db_env_path("dev")
# Get login info.
connection_params = hsql.get_connection_info_from_env_file(env_file)
# Login.
connection = hsql.get_connection(*connection_params)

### Creating tables

In [ ]:
create_all_tables(connection)

### Market data

In [ ]:
# Create an example client connected to DB.
hsql.remove_table(connection, "example2_marketdata")
im_client = icdc.get_mock_realtime_client(connection)
market_data = mdata.get_RealtimeMarketData_example1(im_client)

#### Example of DB data (OHLCV)

In [ ]:
query = "SELECT * FROM example2_marketdata LIMIT 10"
# Execute query and return as pd.DataFrame.
raw_data = hsql.execute_query_to_df(connection, query)
raw_data

### Order example

In [ ]:
order = oordexam.get_order_example4()

## Demonstration of main Broker methods

### Init

In [ ]:
exchange_id = "binance"
universe_version = "v5"
contract_type = "futures"
mode = "test"

In [ ]:
broker = occxbrok.CcxtDbBroker(
    exchange_id,
    universe_version,
    mode,
    contract_type,
    db_connection=connection,
    submitted_orders_table_name=oomsdb.SUBMITTED_ORDERS_TABLE_NAME,
    accepted_orders_table_name=oomsdb.ACCEPTED_ORDERS_TABLE_NAME,
    strategy_id="SAU1",
    market_data=market_data,
)

In [ ]:
# Submitting orders and uploading them to DB and saving to S3 location.
# Using the internal method to avoid the polling timeout.
await broker._submit_orders([order], pd.Timestamp.utcnow())

### Checking submitted orders

In [ ]:
# In the DB.
query = f"SELECT * FROM {oomsdb.SUBMITTED_ORDERS_TABLE_NAME} LIMIT 10"
hsql.execute_query_to_df(connection, query)

In [ ]:
# On the S3 location.
hs3.listdir(
    "s3://cryptokaizen-data-test/ccxt_db_broker_test/",
    "*",
    True,
    False,
    aws_profile="ck",
)

In [ ]:
hs3.from_file(
    "s3://cryptokaizen-data-test/ccxt_db_broker_test/20220704000000/positions.0.20220704_190819.txt",
    aws_profile="ck",
)